In [1]:
import numpy as np

In [2]:
# DAG provided in appendix A
dag = {
    (0, 30), (1, 0), (2, 7), (3, 2), (4, 1), (5, 15), (6, 5), (7, 6), (8, 7), (9, 8), (10, 0), (11, 4), (12, 11), (13, 12), (16, 14), (14, 10), (15, 4), (16, 15), (17, 16), (18, 17), (19, 18), (20, 17), (21, 20), (22, 21), (23, 4), (24, 23), (25, 24), (26, 25), (27, 25), (28, 26), (28, 27), (29, 3), (29, 9),
    (29, 13), (29, 19), (29, 22), (29, 28)
}

# processing times provided in Appendix A
p = [3, 10, 2, 2, 5, 2, 14, 5, 6, 5, 5, 2, 3, 3, 5, 6, 6, 6, 2, 3, 2, 3, 14, 5, 18, 10, 2, 3, 6, 2, 10]

# due dates provided in Appendix A
d = [172, 82, 18, 61, 93, 71, 217, 295, 290, 287, 253, 307, 279, 73, 355, 34, 233, 77, 88, 122, 71, 181, 340, 141, 209, 217, 256, 144, 307, 329, 269]

num_nodes = len(p)

In [3]:
# Constructs an inverted graph by adjacency list and num_successors from a set of edges.
def build_graph(edges):

    inv_graph = dict()
    num_successors = dict()
    
    for u, v in edges:
        if u not in num_successors:
            num_successors[u] = 0
        if v not in num_successors:
            num_successors[v] = 0

        # Add nodes and edges to the graph
        if v not in inv_graph: inv_graph[v] = []
        inv_graph[v].append(u)
        num_successors[u] += 1
        
    return inv_graph, num_successors

# Calculates the tardiness giving a completion time and due date
def tardiness(C, dd): return max(0, C-dd)
    

In [28]:
# Finds an optimal schedule according to LCL given a graph, processing times, and due dates
def least_cost_last(dag, processing_times, due_dates):

    # build adjacency list and track number of successors
    inv_graph, num_successors = build_graph(dag)

    schedule = []
    total_processing_time = sum(processing_times)
    total_cost = 0

    # Jobs to be scheduled are ones with 0 successors
    ready_to_schedule = {job for job in num_successors if num_successors[job] == 0}
    count = 0
    # While we have jobs to be scheduled left
    while ready_to_schedule:
        count += 1
        # Choose the job with the min tardiness
        last_job = min(ready_to_schedule, key=lambda j: tardiness(total_processing_time, due_dates[j]))

        schedule.append(last_job)
        total_cost = max(total_cost, tardiness(total_processing_time, due_dates[last_job]))
        ready_to_schedule.remove(last_job)
        total_processing_time -= processing_times[last_job]
        
        # If the job is a root in the graph
        if last_job not in inv_graph: continue

        for parent in inv_graph[last_job]:

            # Decrement number of successors because we scheduled the job
            num_successors[parent] -= 1

            # If ready to be scheduled, add to list
            if num_successors[parent] == 0:  
                ready_to_schedule.add(parent)
        if count in {1, 2, 5, 10, 15, 20, 25, 30}:
            print('Iteration number {0}: Schedule is '.format(count), [1 + n for n in schedule[::-1]])
    return [1 + n for n in schedule[::-1]], total_cost


In [29]:
least_cost_last(dag, p, d)

Iteration number 1: Schedule is  [30]
Iteration number 2: Schedule is  [0, 30]
Iteration number 5: Schedule is  [1, 14, 10, 0, 30]
Iteration number 10: Schedule is  [11, 25, 24, 23, 4, 1, 14, 10, 0, 30]
Iteration number 15: Schedule is  [13, 28, 12, 27, 26, 11, 25, 24, 23, 4, 1, 14, 10, 0, 30]
Iteration number 20: Schedule is  [19, 18, 17, 16, 15, 13, 28, 12, 27, 26, 11, 25, 24, 23, 4, 1, 14, 10, 0, 30]
Iteration number 25: Schedule is  [7, 6, 21, 5, 20, 19, 18, 17, 16, 15, 13, 28, 12, 27, 26, 11, 25, 24, 23, 4, 1, 14, 10, 0, 30]
Iteration number 30: Schedule is  [9, 8, 3, 2, 22, 7, 6, 21, 5, 20, 19, 18, 17, 16, 15, 13, 28, 12, 27, 26, 11, 25, 24, 23, 4, 1, 14, 10, 0, 30]


([30,
  10,
  9,
  4,
  3,
  23,
  8,
  7,
  22,
  6,
  21,
  20,
  19,
  18,
  17,
  16,
  14,
  29,
  13,
  28,
  27,
  12,
  26,
  25,
  24,
  5,
  2,
  15,
  11,
  1,
  31],
 65)